In [6]:
# libraries
import random
from keras.layers import Dense, Dropout, LSTM
from keras.models import Sequential
import numpy as np
import pandas_gbq as pd
import pickle
import json
import nltk
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.optimizers import Adam
lemmatizer = WordNetLemmatizer()
nltk.download('omw-1.4')
nltk.download("punkt")
nltk.download("wordnet")

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [7]:
# Load intents data
data_file = open("/content/intents.json").read()
intents = json.loads(data_file)

In [8]:
# Preprocess data
words = []
classes = []
documents = []
ignore_words = ["?", "!"]

for intent in intents["intents"]:
    for pattern in intent["patterns"]:
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        documents.append((w, intent["tag"]))
        if intent["tag"] not in classes:
            classes.append(intent["tag"])

words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
classes = sorted(list(set(classes)))

print(len(documents), "documents")
print(len(classes), "classes", classes)
print(len(words), "unique lemmatized words", words)

100 documents
55 classes ['AI', 'abbr', 'artificial', 'bend', 'body', 'bot1', 'breathe', 'business', 'chatbot', 'chatterbox', 'clone', 'comp', 'computer', 'control', 'cramped', 'date', 'death', 'do', 'events', 'fav', 'fight', 'goodbye', 'greetings', 'hardware', 'hobby', 'idea', 'imortal', 'lang', 'laugh', 'lie', 'machine', 'malfunction', 'motormouth', 'move', 'name', 'name1', 'need', 'noanswer', 'os', 'program', 'programming', 'ratchet', 'robotics', 'robots', 'robotss', 'sapient', 'sense', 'sentiment', 'shoe', 'sound', 'stupid', 'thanks', 'usage', 'who', 'wt']
126 unique lemmatized words ["'m", "'s", ',', 'a', 'ai', 'all', 'allowed', 'am', 'an', 'are', 'artificial', 'awesome', 'be', 'being', 'bend', 'body', 'bot', 'breathe', 'business', 'bye', 'can', 'chat', 'chatterbox', 'clone', 'coffee', 'computer', 'control', 'cramped', 'data', 'date', 'die', 'do', 'entity', 'event', 'favorite', 'favour', 'fight', 'for', 'good', 'great', 'hardware', 'haroo', 'hello', 'help', 'helpful', 'helping', '

In [9]:
# Save processed data
pickle.dump(words, open("words.pkl", "wb"))
pickle.dump(classes, open("classes.pkl", "wb"))

In [10]:
# Prepare training data
training = []
output_empty = [0] * len(classes)

for doc in documents:
    pattern_words = doc[0]
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    bag = [1 if w in pattern_words else 0 for w in words]
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])

In [11]:
# Shuffle training data
random.shuffle(training)

In [12]:
# Separate bag-of-words representations and output labels
train_x = [item[0] for item in training]
train_y = [item[1] for item in training]

In [13]:
# Convert to NumPy arrays
train_x = np.array(train_x)
train_y = np.array(train_y)

print("Training data created")

Training data created


In [14]:
# Reshape train_x to add a new axis for the timestep
train_x = np.reshape(train_x, (train_x.shape[0], 1, train_x.shape[1]))

In [16]:
# Create LSTM model
model = Sequential()
model.add(LSTM(64, input_shape=(train_x.shape[1], train_x.shape[2])))  # Input shape with timesteps
model.add(Dense(128, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation="softmax"))
model.summary()

# Compile model
model.compile(loss="categorical_crossentropy", optimizer=Adam(), metrics=["accuracy"])

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 64)                48896     
                                                                 
 dense_3 (Dense)             (None, 128)               8320      
                                                                 
 dropout_2 (Dropout)         (None, 128)               0         
                                                                 
 dense_4 (Dense)             (None, 64)                8256      
                                                                 
 dropout_3 (Dropout)         (None, 64)                0         
                                                                 
 dense_5 (Dense)             (None, 55)                3575      
                                                                 
Total params: 69047 (269.71 KB)
Trainable params: 6904

In [18]:
# Train model
hist = model.fit(train_x, train_y, epochs=400, batch_size=5, verbose=1)

Epoch 1/400
20/20 [==============================] - 2s 5ms/step - loss: 0.1197 - accuracy: 0.9500
Epoch 2/400
20/20 [==============================] - 0s 5ms/step - loss: 0.1103 - accuracy: 0.9600
Epoch 3/400
20/20 [==============================] - 0s 5ms/step - loss: 0.1002 - accuracy: 0.9800
Epoch 4/400
20/20 [==============================] - 0s 5ms/step - loss: 0.1128 - accuracy: 0.9500
Epoch 5/400
20/20 [==============================] - 0s 6ms/step - loss: 0.1604 - accuracy: 0.9500
Epoch 6/400
20/20 [==============================] - 0s 8ms/step - loss: 0.1613 - accuracy: 0.9400
Epoch 7/400
20/20 [==============================] - 0s 7ms/step - loss: 0.1484 - accuracy: 0.9400
Epoch 8/400
20/20 [==============================] - 0s 8ms/step - loss: 0.0723 - accuracy: 0.9600
Epoch 9/400
20/20 [==============================] - 0s 9ms/step - loss: 0.1165 - accuracy: 0.9500
Epoch 10/400
20/20 [==============================] - 0s 8ms/step - loss: 0.1373 - accuracy: 0.9500
Epoch 11/

In [24]:
train_acc = hist.history['accuracy']
trainn_acc = train_acc[-1]

print(f"Training Accuracy: {trainn_acc:.2%}")

Training Accuracy: 96.00%


In [23]:
# Save model
model.save("chatbot_model.h5", hist)
print("Model created")

Model created


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


***Done Here! Thanks***